In [1]:
!pip install amply pyomo -q
!python -m amplpy.modules install coin highs scip gcg -q

In [2]:

from amplpy import modules
import pyomo.environ as pyo
import pandas as pd
import numpy as np
from pyomo.environ import *


import matplotlib.pyplot as plt
import networkx as nx
import requests



# Caso 1

In [3]:
clientes_data = pd.read_csv(r"C:\Users\ACER\OneDrive - Universidad de los Andes\Escritorio\MOS\PROY2_MOS\Proyecto Seneca Libre\case_1_base\Clients.csv")
clientes_data.rename(columns = {"ClientID": "ID"}, inplace = True)
clientes_data['type']= 'Cliente'
print(clientes_data.shape)
clientes_data.sample(5)


(50, 5)


,ID,Product,Longitude,Latitude,type
46,47,14,-74.067974,4.561064,Cliente
45,46,20,-74.142028,4.545515,Cliente
35,36,15,-74.111904,4.693008,Cliente
7,8,14,-74.109241,4.675777,Cliente
18,19,13,-74.109520,4.605100,Cliente


In [4]:
clientes_data['Unique_ID'] = clientes_data.apply(
    lambda row: f"Depot_{row['ID']}" if row['type'] == 'Depot' else f"Client_{row['ID']}",
    axis=1
)
clientes_data.sample(5)

,ID,Product,Longitude,Latitude,type,Unique_ID
16,17,18,-74.130778,4.593030,Cliente,Client_17
12,13,16,-74.089625,4.777614,Cliente,Client_13
4,5,10,-74.108256,4.703706,Cliente,Client_5
40,41,15,-74.034620,4.766478,Cliente,Client_41
34,35,14,-74.100741,4.638657,Cliente,Client_35


In [5]:
depositos_data = pd.read_csv(r"C:\Users\ACER\OneDrive - Universidad de los Andes\Escritorio\MOS\PROY2_MOS\Proyecto Seneca Libre\case_1_base\Depots.csv")
depositos_data['type']= 'Depot'
print(depositos_data.shape)
depositos_data.sample(5)

(12, 4)


,DepotID,Longitude,Latitude,type
2,3,-74.038548,4.792926,Depot
7,8,-74.109756,4.678961,Depot
1,2,-74.109934,4.536383,Depot
9,10,-74.109916,4.550641,Depot
6,7,-74.095619,4.621912,Depot


In [6]:
depositos_data['Unique_ID'] = depositos_data.apply(
    lambda row: f"Depot_{row['DepotID']}" if row['type'] == 'Depot' else f"Client_{row['ID']}",
    axis=1
)
depositos_data.sample(5)

,DepotID,Longitude,Latitude,type,Unique_ID
1,2,-74.109934,4.536383,Depot,Depot_2
2,3,-74.038548,4.792926,Depot,Depot_3
0,1,-74.081242,4.750212,Depot,Depot_1
3,4,-74.067069,4.721678,Depot,Depot_4
8,9,-74.095472,4.735973,Depot,Depot_9


In [7]:
ubicaciones_data = pd.concat([depositos_data[["Unique_ID","Longitude", "Latitude","type"]], clientes_data[["Unique_ID","Longitude", "Latitude", "type"]]], axis = 0)
print(ubicaciones_data.shape)
ubicaciones_data.sample(15)

(62, 4)


,Unique_ID,Longitude,Latitude,type
31,Client_32,-74.029082,4.766433,Cliente
18,Client_19,-74.109520,4.605100,Cliente
20,Client_21,-74.082778,4.644892,Cliente
5,Depot_6,-74.124002,4.650463,Depot
43,Client_44,-74.048092,4.719037,Cliente
39,Client_40,-74.085021,4.770345,Cliente
26,Client_27,-74.108937,4.602030,Cliente
47,Client_48,-74.083275,4.579417,Cliente
40,Client_41,-74.034620,4.766478,Cliente
24,Client_25,-74.097002,4.627605,Cliente


In [8]:
vehiculos = pd.read_csv(r"C:\Users\ACER\OneDrive - Universidad de los Andes\Escritorio\MOS\PROY2_MOS\Proyecto Seneca Libre\case_1_base\multi_vehicles.csv")
print(vehiculos.shape)
vehiculos.sample(5)

(24, 3)


,VehicleType,Capacity,Range
5,Gas Car,108,136
13,EV,73,1075
11,EV,97,715
10,EV,124,954
1,Gas Car,135,195


In [9]:
vehiculos_data= pd.read_csv(r"C:\Users\ACER\OneDrive - Universidad de los Andes\Escritorio\MOS\PROY2_MOS\Proyecto Seneca Libre\vehicles_data.csv")
print(vehiculos_data.shape)
vehiculos_data.sample(3)

(3, 9)


,Vehicle,Freight Rate [COP/km],Time Rate [COP/min],Daily Maintenance [COP/day],Recharge/Fuel Cost [COP/(gal or kWh)],Recharge/Fuel Time [min/10 percent charge],Avg. Speed [km/h],Gas Efficiency [km/gal],Electricity Efficency [kWh/km]
0,Gas Car,5000,500,30000,16000.00,0.1,NaN,10.0,NaN
2,Solar EV,4000,500,21000,NaN,NaN,NaN,NaN,0.15
1,Drone,500,500,3000,220.73,2.0,40.0,NaN,0.15


In [10]:
# DataFrame 2 (datos de vehículos)
vehiculos_data = pd.DataFrame({
    'Vehicle': ['Gas Car', 'EV', 'Drone'],
    'Freight Rate [COP/km]': [5000, 4000, 500],
    'Time Rate [COP/min]': [500, 500, 500],
    'Daily Maintenance [COP/day]': [30000, 21000, 3000],
    'Recharge/Fuel Cost [COP/gal or kWh]': [16000, None, 220.73],
    'Recharge/Fuel Time [min/10 percent charge]': [0.1, None, 2.0],
    'Avg. Speed [km/h]': [None, None, 40.0],
    'Gas Efficiency [km/gal]': [10.0, None, None],
    'Electricity Efficiency [kWh/km]': [None, 0.15, 0.15]
})

# Modificar el DataFrame `vehiculos` para que coincida la nomenclatura de "Solar EV" con "EV"
vehiculos['VehicleType'] = vehiculos['VehicleType'].replace('Solar EV', 'EV')

# Realizar nuevamente el merge entre los DataFrames
vehiculos_info = vehiculos.merge(vehiculos_data, left_on='VehicleType', right_on='Vehicle', how='left')

vehiculos_info.sample(5)

,VehicleType,Capacity,Range,Vehicle,Freight Rate [COP/km],Time Rate [COP/min],Daily Maintenance [COP/day],Recharge/Fuel Cost [COP/gal or kWh],Recharge/Fuel Time [min/10 percent charge],Avg. Speed [km/h],Gas Efficiency [km/gal],Electricity Efficiency [kWh/km]
11,EV,97,715,EV,4000,500,21000,NaN,NaN,NaN,NaN,0.15
12,EV,85,1022,EV,4000,500,21000,NaN,NaN,NaN,NaN,0.15
15,EV,84,1370,EV,4000,500,21000,NaN,NaN,NaN,NaN,0.15
23,Gas Car,120,108,Gas Car,5000,500,30000,16000.0,0.1,NaN,10.0,NaN
1,Gas Car,135,195,Gas Car,5000,500,30000,16000.0,0.1,NaN,10.0,NaN


In [11]:
num_clientes = 10
clientes_id = RangeSet(1, num_clientes)

In [12]:
num_vehiculos = 5
vehiculos_r = RangeSet(1, num_vehiculos)

In [13]:
num_tipo_v = 2
tipos_vehiculo = RangeSet(1, num_tipo_v)

In [14]:
num_depositos = 3
deposito_origen = RangeSet(1, num_depositos)  # Definir ciudad de origen


In [15]:
num_ubicaciones = num_clientes + num_depositos
ubicaciones = RangeSet(1, num_ubicaciones)


In [16]:
def distancia(coor1, coor2):
    service = "route"
    version = "v1"
    profile = "driving"
    coordinates = f"{coor1[0]},{coor1[1]};{coor2[0]},{coor2[1]}"
    overview = "false"

    url = f"http://router.project-osrm.org/{service}/{version}/{profile}/{coordinates}?alternatives=false&overview={overview}"

    try:
        response = requests.get(url)
    except:
        return np.random.randint(1, 5)
    
    if response.status_code == 200:
        return float(response.json()["routes"][0]["distance"])/1000
    else:
        return 99999

In [17]:
from concurrent.futures import ThreadPoolExecutor
def calcular_costos_distancia(num_ubicaciones, ubicaciones_data, ubicaciones):
    costos_distancia = np.zeros((num_ubicaciones, num_ubicaciones))
    coor = [(ubicaciones_data["Longitude"].to_numpy()[i - 1], 
             ubicaciones_data["Latitude"].to_numpy()[i - 1]) for i in ubicaciones]

    # Función para calcular la distancia para un par específico
    def calcular_par_distancia(indices):
        i, j = indices
        return i, j, distancia(coor[i], coor[j])

    # Generar todas las combinaciones de índices
    indices = [(i, j) for i in range(num_ubicaciones) for j in range(num_ubicaciones)]

    # Usar ThreadPoolExecutor para calcular distancias en paralelo
    with ThreadPoolExecutor() as executor:
        resultados = list(executor.map(calcular_par_distancia, indices))

    # Rellenar la matriz con los resultados
    for i, j, dist in resultados:
        costos_distancia[i, j] = dist

    return costos_distancia

In [18]:
costos_distancia = calcular_costos_distancia(num_ubicaciones, ubicaciones_data, ubicaciones)
costos_distancia

array([[9.99990e+04, 9.99990e+04, 1.01619e+01, 6.45240e+00, 2.14157e+01,
        1.69241e+01, 9.99990e+04, 1.13612e+01, 4.32990e+00, 3.12899e+01,
        1.36590e+01, 9.99990e+04, 1.47593e+01],
       [3.29469e+01, 0.00000e+00, 9.99990e+04, 9.99990e+04, 1.39168e+01,
        9.99990e+04, 9.99990e+04, 9.99990e+04, 9.99990e+04, 9.99990e+04,
        2.00000e+00, 2.00000e+00, 2.00000e+00],
       [4.00000e+00, 4.00000e+00, 3.00000e+00, 1.00000e+00, 2.00000e+00,
        3.00000e+00, 3.00000e+00, 2.00000e+00, 3.00000e+00, 1.00000e+00,
        3.00000e+00, 4.00000e+00, 1.00000e+00],
       [2.00000e+00, 1.00000e+00, 4.00000e+00, 1.00000e+00, 4.00000e+00,
        4.00000e+00, 2.00000e+00, 1.00000e+00, 4.00000e+00, 1.00000e+00,
        4.00000e+00, 2.00000e+00, 3.00000e+00],
       [3.00000e+00, 2.00000e+00, 1.00000e+00, 2.00000e+00, 4.00000e+00,
        4.00000e+00, 2.00000e+00, 4.00000e+00, 2.00000e+00, 2.00000e+00,
        4.00000e+00, 4.00000e+00, 3.00000e+00],
       [3.00000e+00, 4.00000e+

In [19]:
def duracion(coor1, coor2):
    service = "route"
    version = "v1"
    profile = "driving"
    coordinates = f"{coor1[0]},{coor1[1]};{coor2[0]},{coor2[1]}"
    overview = "false"

    url = f"http://router.project-osrm.org/{service}/{version}/{profile}/{coordinates}?alternatives=false&overview={overview}"

    try:
        response = requests.get(url)
    except:
        return np.random.randint(1, 5)
    
    if response.status_code == 200:
        return float(response.json()["routes"][0]["duration"])/60
    else:
        return 99999

In [20]:
from concurrent.futures import ThreadPoolExecutor
def calcular_costos_duracion(num_ubicaciones, ubicaciones_data, ubicaciones):
    costos_duracion = np.zeros((num_ubicaciones, num_ubicaciones))
    coor = [(ubicaciones_data["Longitude"].to_numpy()[i - 1], 
             ubicaciones_data["Latitude"].to_numpy()[i - 1]) for i in ubicaciones]

    # Función para calcular la duracion para un par específico
    def calcular_par_duracion(indices):
        i, j = indices
        return i, j, duracion(coor[i], coor[j])

    # Generar todas las combinaciones de índices
    indices = [(i, j) for i in range(num_ubicaciones) for j in range(num_ubicaciones)]

    # Usar ThreadPoolExecutor para calcular duraciones en paralelo
    with ThreadPoolExecutor() as executor:
        resultados = list(executor.map(calcular_par_duracion, indices))

    # Rellenar la matriz con los resultados
    for i, j, dura in resultados:
        costos_duracion[i, j] = dura

    return costos_duracion

In [21]:
costos_duracion = calcular_costos_duracion(num_ubicaciones, ubicaciones_data, ubicaciones)
costos_duracion

array([[4.00000000e+00, 2.00000000e+00, 3.00000000e+00, 3.00000000e+00,
        1.00000000e+00, 2.00000000e+00, 2.00000000e+00, 1.00000000e+00,
        2.00000000e+00, 1.00000000e+00, 4.00000000e+00, 4.00000000e+00,
        3.00000000e+00],
       [3.00000000e+00, 2.00000000e+00, 2.00000000e+00, 1.00000000e+00,
        4.00000000e+00, 3.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        4.00000000e+00, 2.00000000e+00, 2.00000000e+00, 3.00000000e+00,
        2.00000000e+00],
       [2.00000000e+00, 1.00000000e+00, 3.00000000e+00, 3.00000000e+00,
        4.00000000e+00, 1.00000000e+00, 1.00000000e+00, 3.00000000e+00,
        4.00000000e+00, 1.00000000e+00, 3.00000000e+00, 3.00000000e+00,
        4.00000000e+00],
       [4.00000000e+00, 1.00000000e+00, 3.00000000e+00, 2.00000000e+00,
        2.00000000e+00, 2.00000000e+00, 2.00000000e+00, 4.00000000e+00,
        4.00000000e+00, 1.00000000e+00, 1.00000000e+00, 3.00000000e+00,
        3.00000000e+00],
       [4.00000000e+00, 4.00000000e+

In [22]:
from pyomo.environ import *
import pandas as pd
import numpy as np

In [23]:
model = pyo.ConcreteModel()

In [24]:
# Eliminar los componentes existentes antes de redefinirlos
if "N" in model.component_map():
    model.del_component("N")
if "C" in model.component_map():
    model.del_component("C")
if "D" in model.component_map():
    model.del_component("D")

In [25]:
model.C = Set(initialize=clientes_data['Unique_ID'].tolist())  # Clientes
model.D = Set(initialize=depositos_data['Unique_ID'].tolist())  # Depósitos
vehiculos_info["Vehicle_ID"] = vehiculos_info.index.astype(str)  # Crear un identificador único si no existe
model.V = Set(initialize=vehiculos_info["Vehicle_ID"].tolist())
model.N = Set(initialize=list(model.C) + list(model.D)) # Nodos



In [26]:
print("Clientes:", model.C.pprint())
print("Depósitos:", model.D.pprint())
print("Nodos (N):", model.N.pprint())


C : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :   50 : {'Client_1', 'Client_2', 'Client_3', 'Client_4', 'Client_5', 'Client_6', 'Client_7', 'Client_8', 'Client_9', 'Client_10', 'Client_11', 'Client_12', 'Client_13', 'Client_14', 'Client_15', 'Client_16', 'Client_17', 'Client_18', 'Client_19', 'Client_20', 'Client_21', 'Client_22', 'Client_23', 'Client_24', 'Client_25', 'Client_26', 'Client_27', 'Client_28', 'Client_29', 'Client_30', 'Client_31', 'Client_32', 'Client_33', 'Client_34', 'Client_35', 'Client_36', 'Client_37', 'Client_38', 'Client_39', 'Client_40', 'Client_41', 'Client_42', 'Client_43', 'Client_44', 'Client_45', 'Client_46', 'Client_47', 'Client_48', 'Client_49', 'Client_50'}
Clientes: None
D : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :   12 : {'Depot_1', 'Depot_2', 'Depot_3', 'Depot_4', 'Depot_5', 'Depot_6', 'Depot_7', 'Depot_8', 'Depot_9', 'De

In [27]:
vehiculos_info.sample(10)

,VehicleType,Capacity,Range,Vehicle,Freight Rate [COP/km],Time Rate [COP/min],Daily Maintenance [COP/day],Recharge/Fuel Cost [COP/gal or kWh],Recharge/Fuel Time [min/10 percent charge],Avg. Speed [km/h],Gas Efficiency [km/gal],Electricity Efficiency [kWh/km],Vehicle_ID
9,Gas Car,98,107,Gas Car,5000,500,30000,16000.0,0.1,NaN,10.0,NaN,9
0,Gas Car,132,145,Gas Car,5000,500,30000,16000.0,0.1,NaN,10.0,NaN,0
23,Gas Car,120,108,Gas Car,5000,500,30000,16000.0,0.1,NaN,10.0,NaN,23
6,Gas Car,125,92,Gas Car,5000,500,30000,16000.0,0.1,NaN,10.0,NaN,6
22,EV,84,1070,EV,4000,500,21000,NaN,NaN,NaN,NaN,0.15,22
21,Gas Car,120,152,Gas Car,5000,500,30000,16000.0,0.1,NaN,10.0,NaN,21
3,Gas Car,157,173,Gas Car,5000,500,30000,16000.0,0.1,NaN,10.0,NaN,3
1,Gas Car,135,195,Gas Car,5000,500,30000,16000.0,0.1,NaN,10.0,NaN,1
17,Gas Car,120,116,Gas Car,5000,500,30000,16000.0,0.1,NaN,10.0,NaN,17
7,Gas Car,78,133,Gas Car,5000,500,30000,16000.0,0.1,NaN,10.0,NaN,7


In [28]:
# Parámetros ajustados
model.capacity = pyo.Param(model.V, initialize=dict(zip(vehiculos_info["Vehicle_ID"], vehiculos_info["Capacity"])))
model.range = pyo.Param(model.V, initialize=dict(zip(vehiculos_info["Vehicle_ID"], vehiculos_info["Range"])))
model.cost_per_km = pyo.Param(model.V, initialize=dict(zip(vehiculos_info["Vehicle_ID"], vehiculos_info["Freight Rate [COP/km]"])))
model.cost_per_min = pyo.Param(model.V, initialize=dict(zip(vehiculos_info["Vehicle_ID"], vehiculos_info["Time Rate [COP/min]"])))
# Parámetro de costo por unidad de carga transportada (por ejemplo, COP por kg)
model.cost_per_load = pyo.Param(initialize=100)


# Demanda por cliente
model.demand = pyo.Param(model.C, initialize=dict(zip(clientes_data["Unique_ID"], clientes_data["Product"])))

# Costos por distancia
dist_dict = {
    (ubicaciones_data.iloc[i]["Unique_ID"], ubicaciones_data.iloc[j]["Unique_ID"]): costos_distancia[i, j]
    for i in range(num_ubicaciones)
    for j in range(num_ubicaciones)
    if i != j
}

model.distances = pyo.Param(model.N, model.N, initialize=dist_dict, default=0)

# Costos por duración
dura_dict = {
    (ubicaciones_data.iloc[i]["Unique_ID"], ubicaciones_data.iloc[j]["Unique_ID"]): costos_duracion[i, j]
    for i in range(num_ubicaciones)
    for j in range(num_ubicaciones)
    if i != j
}

model.durations = pyo.Param(model.N, model.N, initialize=dura_dict, default=0)

# Costo de mantenimiento diario de cada vehículo
model.maintenance_cost = pyo.Param(model.V, initialize=dict(zip(vehiculos_info["Vehicle_ID"], vehiculos_info["Daily Maintenance [COP/day]"])))


In [29]:
# Variables binarias para rutas
model.x = pyo.Var(model.N, model.N, model.V, domain=pyo.Binary)

# Variables de carga acumulada
model.carga_load = pyo.Var(model.N, model.V, domain=pyo.NonNegativeReals)

# Variable auxiliar para el orden de visita
model.u = pyo.Var(model.V, model.N, within=pyo.NonNegativeIntegers)



In [30]:
# Función objetivo ajustada
def objective_rule(model):
    # Costo por distancia recorrida
    cost_distance = sum(model.cost_per_km[v] * model.distances[i, j] * model.x[i, j, v]
                        for v in model.V for i in model.N for j in model.N if i != j)
    
    # Costo por duración del trayecto
    cost_duration = sum(model.cost_per_min[v] * model.durations[i, j] * model.x[i, j, v]
                        for v in model.V for i in model.N for j in model.N if i != j)
    
    # Costo de mantenimiento diario
    cost_maintenance = sum(model.maintenance_cost[v] for v in model.V)
    
     # Costo por carga transportada
    cost_load = sum(model.cost_per_load * model.carga_load[i, v] for v in model.V for i in model.C)
    
    # Suma total de costos
    return cost_distance + cost_duration + cost_maintenance + cost_load

model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

In [ ]:
#RESTRICCIONES
#Quitar carga load y arreglar la restriccion de capacidad de los vehiculos
# Restricciones de vista una vez
def visit_once_rule(model, j):
    return sum(model.x[i, j, v] for v in model.V for i in model.N if i != j) == 1

model.visit_once = pyo.Constraint(model.C, rule=visit_once_rule)

#Restricción conservación de flujo
def flow_conservation_rule(model, v, i):
    return sum(model.x[i, j, v] for j in model.N if i != j) == sum(model.x[j, i, v] for j in model.N if i != j)

model.flow_conservation = pyo.Constraint(model.V, model.N, rule=flow_conservation_rule)


# Restricción para satisfacer la demanda de cada cliente
def satisfy_demand_rule(model, i):
    return sum(model.capacity[v]*model.x[i, j, v] for j in model.N for v in model.V if i != j) >= model.demand[i]
model.satisfy_demand = pyo.Constraint(model.C, rule=satisfy_demand_rule)


# Restricción de capacidad de los vehículos
def vehicle_capacity_rule(model, i, j, v):
    if i in model.C:
        return model.carga_load[j, v] >= model.carga_load[i, v] + model.demand[i] * model.x[i, j, v] - model.capacity[v] * (1 - model.x[i, j, v])
    else:
        return pyo.Constraint.Skip

model.vehicle_capacity = pyo.Constraint(model.N, model.N, model.V, rule=vehicle_capacity_rule)

# Restricción de subtour elimination (MTZ)
def subtour_elimination_rule(model, v, i, j):
    if i != j:  # Aplica a nodos diferentes
        return model.u[v, i] - model.u[v, j] + len(model.N) * model.x[i, j, v] <= len(model.N) - 1
    else:
        return pyo.Constraint.Skip

model.subtour_elimination = pyo.Constraint(model.V, model.N, model.N, rule=subtour_elimination_rule)

# Restricción para garantizar que los vehículos regresen al depósito de salida
def return_to_depot_rule(model, v, d):
    return sum(model.x[d, j, v] for j in model.N if j != d) == sum(model.x[i, d, v] for i in model.N if i != d)

model.return_to_depot = pyo.Constraint(model.V, model.D, rule=return_to_depot_rule)

def depot_return_rule(model, v):
    return sum(model.x[d, j, v] for d in model.D for j in model.C) == 1


#Restricción salida de deposito
model.depot_return = pyo.Constraint(model.V, rule=depot_return_rule)




In [40]:
class ModelSolver:
    def __init__(self, model):
        self.model = model

    def solve_model(self):
        # Definir el solver
        solver_name = "highs"  # Asegúrate de tener este solver instalado y disponible
        solver = pyo.SolverFactory(solver_name+"nl", executable=modules.find(solver_name), solver_io="nl")

        # Configurar opciones avanzadas del solver
        solver.options['parallel'] = 'on'  # Habilitar paralelismo
        solver.options['time_limit'] = 3600/2  # Límite de tiempo (1 hora)
        solver.options['presolve'] = 'on'  # Habilitar preprocesamiento
        solver.options['mip_rel_gap'] = 0.01  # Gap relativo del 1% (para MIP)
        solver.options['simplex_strategy'] = 1  # Estrategia del método simplex (dual simplex)
        #solver.options['simplex_max_concurrency'] = 8  # Máxima concurrencia para simplex
        #solver.options['mip_min_logging_interval'] = 10  # Intervalo mínimo de logs (10s)
        #solver.options['mip_heuristic_effort'] = 0.2  # Esfuerzo heurístico en 20%

        # Resolver el modelo
        result = solver.solve(self.model, tee=True)
        print(result.solver.status)
        print(result.solver.termination_condition)

        # Verificar el estado de la solución
        if result.solver.termination_condition == pyo.TerminationCondition.optimal:
            print("Optimal solution found.")
        elif result.solver.termination_condition == pyo.TerminationCondition.maxTimeLimit:
            print("Time limit reached, solution may be suboptimal.")
        else:
            print(f"Solver terminated with condition: {result.solver.termination_condition}")

        # Mostrar el resultado
        print(result)


In [41]:
solver = ModelSolver(model)
solver.solve_model()

HiGHS 1.8.1:   alg:parallel = on
  lim:time = 1800
  pre:solve = on
  mip:gap = 0.01
  alg:simplex = 1
  Warning code 1 for call Highs_run(lp())
model.name="unknown";
    - termination condition: maxIterations
    - message from solver: HiGHS 1.8.1\x3a time limit, no solution; 357537
      simplex iterations; 235 branching nodes
warning
maxIterations
Solver terminated with condition: maxIterations

Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 167068
  Number of variables: 0
  Sense: unknown
Solver: 
- Status: warning
  Message: HiGHS 1.8.1\x3a time limit, no solution; 357537 simplex iterations; 235 branching nodes
  Termination condition: maxIterations
  Id: 472
  Error rc: 0
  Time: 1801.62824010849
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [ ]:
solver_name="highs"

In [ ]:
opt = pyo.SolverFactory(solver_name+"nl", executable=modules.find(solver_name), solver_io="nl")
results = opt.solve(model, tee=True)
model.display()

In [ ]:

# Extraer las rutas seleccionadas
selected_routes = []
for k in model.V:
    routes_for_k = [(i, j) for i in ubicaciones for j in ubicaciones if i != j and model.x[i, j, k].value == 1]
    print(f"Rutas seleccionadas para el viajero {k}: {routes_for_k}")
    selected_routes.append(routes_for_k)

# Crear el grafo usando NetworkX
G = nx.DiGraph()

# Añadir ubicaciones
for i in ubicaciones:
    G.add_node(i)

for k in range(num_vehiculos):
    for route in selected_routes[k]:
        i, j = route
        G.add_edge(i, j, weight=f"{costos_distancia[i-1,j-1]:.2f}",color=k)

# Dibujar el grafo
edges = G.edges()
edge_colors = [G[u][v]['color'] for u, v in edges]
pos = nx.planar_layout(G)  # Usar un layout circular para una visualización clara
labels = nx.get_edge_attributes(G, 'weight')

plt.figure(figsize=(8, 8))
nx.draw(G, pos, with_labels=True, node_color='lightblue', node_size=100, font_size=5, font_weight='bold', arrowsize=20, edge_color=edge_colors)
nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)

plt.title("Rutas Óptimas de Inspección de Infraestructura con 5 Equipos", size=20)
plt.show()
pos

KeyError: "Index '(1, 2, '0')' is not valid for indexed component 'x'"

: 